In [63]:
# core scientific libraries
import os
import sys
import numpy as np
import pandas as pd
import xarray as xr
import datetime

# plotting
import matplotlib.pyplot as plt
import seaborn as sns
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# secondary libraries
import netCDF4 as nc
from wrf import (to_np, getvar, smooth2d, get_cartopy, cartopy_xlim,
                 cartopy_ylim, latlon_coords, interplevel, CoordPair, vertcross, g_uvmet, interpline, destagger,
                interp2dxy, ll_to_xy, xy, ALL_TIMES, vinterp, uvmet)
import metpy.calc as mpcalc
from metpy.units import units
import metpy.constants as constants
from math import fabs, log, cos, sin, tan, pi

import pytz

from xarray.backends.netCDF4_ import NetCDF4DataStore

import copy 
import time

In [64]:
# Set filepath
datdir = '../input_data/'
filename = 'wrfout_d03_2017-05-22_00:00:00'

# set output filepath
outdir = '/home/sbarc/students/coello/repos/classes/spring_2021/geog_288cj/post_processing/outputs/'
fig_outdir = '/home/sbarc/students/coello/repos/classes/spring_2021/geog_288cj/post_processing/outputs/figures/'
dat_outdir = '/home/sbarc/students/coello/repos/classes/spring_2021/geog_288cj/post_processing/outputs/data/'


# open wrf Dataset object
filepath = datdir + filename
print('Opened File:', filepath, '\n')

# Open NetCDF Dataset object
f = nc.Dataset(filepath,'r')

# List variable names
#print("Variables:")
#print(f.variables.keys(),'\n')

# Show variavles and all details
#print(f.variables.items(),'\n')

# Show dimension names and sizes
#print("Dimensions:")
#print(f.dimensions.items(),'\n')

# Print all file metadata
#print(f)   # or run 'ncdump -h <filename>' in terminal

Opened File: ../input_data/wrfout_d03_2017-05-22_00:00:00 



In [65]:
# specify desired variables to grad from the wrf out file and their units
des_vars = ["T2", "slp", "uvmet10", "uvmet10_wspd_wdir", "rh2"]
var_units = [None, "hpa", "m s-1", "m s-1", None]

# specify the z levels to interpolate to [km]
v_levels = np.arange(0, 21, 0.05)

# specify the time index desired
indx = ALL_TIMES

# make empty list to append interpolated dataarrays too
data = []

# start timer
t0 = time.time()

for i in range(len(des_vars)):
    
    # grab var with units at time index
    if (var_units[i] != None):
        tmp = getvar(f, des_vars[i], units = var_units[i], timeidx = indx)
    else:
        tmp = getvar(f, des_vars[i], timeidx = indx)
    
    # interpolate to z levels
    if (des_vars[i] == "uvmet10" or des_vars[i] == "uvmet10_wspd_wdir"): # some get vars output two variables
        tmp_1, tmp_2 = tmp
        
        if (des_vars[i] == "uvmet10"):
            tmp_1 = tmp_1.rename('u').drop("u_v")
            tmp_2 = tmp_2.rename('v').drop("u_v")
        else:
            tmp_1 = tmp_1.rename('wspd').drop("wspd_wdir")
            tmp_2 = tmp_2.rename('wdir').drop("wspd_wdir")
         
        print("Opened Variable:" + des_vars[i])

        # fix projection labeling for saving as a netcdf https://github.com/NCAR/wrf-python/issues/91
        del tmp_1.attrs['coordinates']
        del tmp_2.attrs['coordinates']
        
        tmp_1.attrs['projection'] = str(tmp_1.attrs['projection'])
        tmp_2.attrs['projection'] = str(tmp_2.attrs['projection'])

        # append to list
        data.append(tmp_1)
        data.append(tmp_2)

        
    else:
        print("Opened Variable:" + des_vars[i])

        # fix projection labeling for saving as a netcdf https://github.com/NCAR/wrf-python/issues/91
        del tmp.attrs['coordinates']

        tmp.attrs['projection'] = str(tmp.attrs['projection'])

        # append to list
        data.append(tmp)
    
    
# merge into one dataset    
data = xr.merge(data)

# add terrian height to dataset after correcting the projection labeling
terr = getvar(f, "ter", units = 'm', timeidx = 0)

del terr.attrs['coordinates']

terr.attrs['projection'] = str(terr.attrs['projection'])
 
data['terr']  = terr  
    
# pull lat and lon vallues and replace current corrdinates
lats = data.XLAT.values[:,0]
lons = data.XLONG.values[0,:]

data['south_north'] = lats
data['west_east'] = lons

# remane dimensions with ugly names
data = data.rename({'south_north':'lat', 'west_east':'lon', 'XTIME':'time', 'XLAT':'lat_grid', 
                    'XLONG':'lon_grid'})

# fix the time coordinates
data['time'] = data['Time']
data = data.drop("Time")

# end timer
t1 = time.time()
total = t1-t0
print('----------------------')
print("time:", total)

Opened Variable:T2
Opened Variable:slp
Opened Variable:uvmet10
Opened Variable:uvmet10_wspd_wdir
Opened Variable:rh2
----------------------
time: 19.1366126537323


In [66]:
# convert temp from K to c
data['T2'] = data['T2'] - 273.15

In [67]:
filename_out = filename + "_surf"

data.to_netcdf(path = dat_outdir + filename_out)

print('Saved Files As: \n', dat_outdir + filename_out)

Saved Files As: 
 /home/sbarc/students/coello/repos/classes/spring_2021/geog_288cj/post_processing/outputs/data/wrfout_d03_2017-05-22_00:00:00_surf
